# Rockets, rocket engines - Homework 1

### Imports

In [113]:
import numpy as np
import pandas as pd

### Load datas

In [114]:
stage_df = pd.read_csv('../data/import/data-1.csv')
display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s
0,1,458900,428300,2830
1,2,168300,157300,3140
2,3,50100,46600,3190


In [115]:
payload_df = pd.read_csv('../data/import/data-2.csv')
display(payload_df)
payload_mass = payload_df.loc[payload_df["component"] == "payload", "m_t"].values[0]
print(payload_mass)

,component,m_t
0,payload,22000


22000


## 1. How much Δ𝑣 accelaration can be achived with each stage?

In [116]:
def delta_v(we, m_0, m_f):
     return we * np.log(m_0 / m_f)

In [117]:
stage_df["m_struct_t"] = stage_df["m_0_t"] - stage_df["m_prop_t"]

In [118]:
display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t
0,1,458900,428300,2830,30600
1,2,168300,157300,3140,11000
2,3,50100,46600,3190,3500


In [119]:
upper_stages_masses = []
for i in range(len(stage_df)):
    upper_stages_masses.append(sum(stage_df.loc[i+1:, "m_0_t"]))

In [120]:
display(upper_stages_masses)

[218400, 50100, 0]

In [121]:
for i in range(len(stage_df)):
    stage_df.loc[i, "m_final_t"] = upper_stages_masses[i] + payload_mass + stage_df.loc[i, "m_struct_t"]

display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t,m_final_t
0,1,458900,428300,2830,30600,271000.0
1,2,168300,157300,3140,11000,83100.0
2,3,50100,46600,3190,3500,25500.0


In [122]:
for i in range(len(stage_df)):
    stage_df.loc[i, "delta_v"] = delta_v(stage_df.loc[i, "vel_out_m_s"], stage_df.loc[i, "m_0_t"], stage_df.loc[i, "m_final_t"])

display(stage_df)

,stage,m_0_t,m_prop_t,vel_out_m_s,m_struct_t,m_final_t,delta_v
0,1,458900,428300,2830,30600,271000.0,1490.599206
1,2,168300,157300,3140,11000,83100.0,2215.908674
2,3,50100,46600,3190,3500,25500.0,2154.342753


In [123]:
display(stage_df['delta_v']) # m/s

0    1490.599206
1    2215.908674
2    2154.342753
Name: delta_v, dtype: float64

In [124]:
m_sum_prop_with_struct = stage_df["m_prop_t"].sum() + stage_df["m_struct_t"].sum()
print(f"Total mass with propellant {m_sum_prop_with_struct} kg") # kg
print(f"Total mass with propellant {m_sum_prop_with_struct / 1000} t") # t

Total mass with propellant 677300 kg
Total mass with propellant 677.3 t


## 2. Structural coefficient

In [125]:
def structural_coefficient(m_prop_param, m_struct_param):
    return m_struct_param / (m_struct_param + m_prop_param)

In [126]:
stage_df["struct_coeff"] = 0.0

In [127]:
for i in range(len(stage_df)):
    stage_df.loc[i, "struct_coeff"] = structural_coefficient(stage_df.loc[i, "m_prop_t"], stage_df.loc[i, "m_struct_t"])

In [128]:
display(stage_df[["stage", "struct_coeff"]])

,stage,struct_coeff
0,1,0.066681
1,2,0.065359
2,3,0.069860


## 3. Final velocity

In [129]:
delta_v_sum = stage_df["delta_v"].sum()
display(delta_v_sum)

np.float64(5860.8506337930385)

In [130]:
v_earth = 465 # m/s
deg = 45.95

In [131]:
v_final = delta_v_sum + v_earth * np.cos(np.radians(deg))

In [132]:
print(f"Final velocity: {v_final} m/s")
print(f"Final velocity: {v_final / 1000} km/s")

Final velocity: 6184.158553238991 m/s
Final velocity: 6.184158553238991 km/s


## 4. Height

In [133]:
g = 9.81
print(v_final)

6184.158553238991


In [134]:
h = v_final**2 / (2 * g)
print(f"Final height: {h} m")
print(f"Final height: {h / 1000} km")

Final height: 1949226.1473801716 m
Final height: 1949.2261473801716 km


# Single stage javítás

In [135]:
w_s_stage = 3190.0 # m/s
z = np.exp(delta_v_sum / w_s_stage)
print(f"z: {z}")

z: 6.279292104874458


In [136]:
m0_single_stage = z * payload_mass
print(f"Single stage mass: {m0_single_stage} kg")
print(f"Single stage mass: {m0_single_stage / 1000} t")

struct_to_prop_ratio_ref = stage_df["m_struct_t"].sum() / stage_df["m_prop_t"].sum()
print(f"Structural to propellant ratio: {struct_to_prop_ratio_ref}")

m_prop_single_stage = (m0_single_stage -payload_mass) / (1 + struct_to_prop_ratio_ref)
print(f"Propellant mass: {m_prop_single_stage} kg")
print(f"Propellant mass: {m_prop_single_stage / 1000} t")

m_struct_single_stage = m0_single_stage - payload_mass - m_prop_single_stage
print(f"Structural mass: {m_struct_single_stage} kg")
print(f"Structural mass: {m_struct_single_stage / 1000} t")

m_struct_3_stage = stage_df["m_struct_t"].sum()
print(f"3 stage structural mass: {m_struct_3_stage} kg")
print(f"3 stage structural mass: {m_struct_3_stage / 1000} t")

ratio =  m_struct_single_stage / m_struct_3_stage
print(f"Ratio: {ratio}")

Single stage mass: 138144.42630723809 kg
Single stage mass: 138.1444263072381 t
Structural to propellant ratio: 0.0713381841189497
Propellant mass: 108410.6102339228 kg
Propellant mass: 108.4106102339228 t
Structural mass: 7733.816073315291 kg
Structural mass: 7.733816073315291 t
3 stage structural mass: 45100 kg
3 stage structural mass: 45.1 t
Ratio: 0.17148150938614837
